In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# EfficientNet (pip install efficientnet-pytorch)
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm  # 학습 진행률 표시

# ==========================
# ✅ SEED 설정 (재현성 확보)
# ==========================
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==============================
# ✅ Early Stopping 클래스
# ==============================
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ==========================
# ✅ ASPP Block 정의 (Global Branch 포함)
# ==========================
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ==========================
# ✅ EfficientNet + VGG + ASPP 모델 정의
# ==========================
class EfficientNetVGGASPPHybrid(nn.Module):
    def __init__(self, num_classes=6):
        super(EfficientNetVGGASPPHybrid, self).__init__()

        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)  

        aspp_features = self.aspp(fused_features)

        aspp_features = aspp_features.view(aspp_features.size(0), -1)
        output = self.classifier(aspp_features)
        return output

# ==========================
# ✅ 데이터 로딩
# ==========================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# ==========================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetVGGASPPHybrid(num_classes=6).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프 (Epochs 포함)
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")



In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# EfficientNet (pip install efficientnet-pytorch)
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm  # 학습 진행률 표시

# ==========================
# ✅ SEED 설정 (재현성 확보)
# ==========================
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==============================
# ✅ Early Stopping 클래스
# ==============================
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ==========================
# ✅ ASPP Block 정의 (Global Branch 포함)
# ==========================
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1_for_global(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ==========================
# ✅ EfficientNet + VGG + ASPP 모델 정의
# ==========================
class EfficientNetVGGASPPHybrid(nn.Module):
    def __init__(self, num_classes=6):
        super(EfficientNetVGGASPPHybrid, self).__init__()

        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)  

        aspp_features = self.aspp(fused_features)

        aspp_features = aspp_features.view(aspp_features.size(0), -1)
        output = self.classifier(aspp_features)
        return output

# ==========================
# ✅ 데이터 로딩
# ==========================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# ==========================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetVGGASPPHybrid(num_classes=6).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프 (Epochs 포함)
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")



Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 3.5975 | Train Acc: 0.4011 | Val Loss: 2.0866 | Val Acc: 0.3842
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 1.3336 | Train Acc: 0.5975 | Val Loss: 1.7868 | Val Acc: 0.6441
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 0.9707 | Train Acc: 0.7980 | Val Loss: 0.9054 | Val Acc: 0.8362
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 0.8315 | Train Acc: 0.8559 | Val Loss: 1.0358 | Val Acc: 0.7740

[Epoch 5/100]


...

Train Loss: 0.5101 | Train Acc: 0.9929 | Val Loss: 0.7516 | Val Acc: 0.8983

[Epoch 14/100]


Train Loss: 0.5171 | Train Acc: 0.9915 | Val Loss: 0.7578 | Val Acc: 0.8644

[Epoch 15/100]


Train Loss: 0.5097 | Train Acc: 0.9901 | Val Loss: 0.8005 | Val Acc: 0.8418

[Epoch 16/100]


Train Loss: 0.4961 | Train Acc: 0.9958 | Val Loss: 0.7715 | Val Acc: 0.9040
  [*] Best model saved.

[Epoch 17/100]


Train Loss: 0.4864 | Train Acc: 0.9972 | Val Loss: 0.7894 | Val Acc: 0.9040
Early Stopping Triggered!

Best Accuracy: 0.9040


In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm  # 학습 진행률 표시

class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # ⭐ global_avg도 256채널로 맞춰주는 Conv
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)

        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        # (batch, 2048, 1, 1) → (batch, 256, 1, 1)
        global_avg = self.conv1_for_global(global_avg)
        # (batch, 256, 1, 1) → upsample → (batch, 256, 7, 7)
        global_avg = nn.functional.interpolate(
            global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True
        )

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        out = self.conv1x1_out(out)
        return out


class EfficientNetVGGASPPHybrid(nn.Module):
    def __init__(self, num_classes=6, print_shapes=True):
        super(EfficientNetVGGASPPHybrid, self).__init__()
        self.print_shapes = print_shapes  # 디버깅 여부 설정

        # ✅ EfficientNet-b0
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        # ✅ VGG16
        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        # ✅ Feature Map 크기 조정
        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  # VGG16 마지막 Conv 출력

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        # ✅ 1x1 Conv 적용하여 채널 맞추기
        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)  # 1280 → 1024
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)  # 512 → 1024

        # ✅ ASPP 블록
        self.aspp = ASPP(in_channels=2048, out_channels=256)  # 채널 유지

        # ✅ Classifier
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        if self.print_shapes:
            print(f"\n[Input] {x.shape}")  # ✅ 입력 이미지 크기

        # ✅ EfficientNet Feature Extraction
        eff_features = self.efficientnet.extract_features(x)
        if self.print_shapes:
            print(f"EfficientNet output: {eff_features.shape}")  # 예상: (batch, 1280, 7, 7)

        eff_features = self.efficientnet_pool(eff_features)
        if self.print_shapes:
            print(f"EfficientNet pooled: {eff_features.shape}")  # 예상: (batch, 1280, 7, 7)

        eff_features = self.eff_conv1x1(eff_features)
        if self.print_shapes:
            print(f"EfficientNet 1x1 Conv: {eff_features.shape}")  # 예상: (batch, 1024, 7, 7)

        # ✅ VGG16 Feature Extraction
        vgg_features = self.vgg.features(x)
        if self.print_shapes:
            print(f"VGG16 output: {vgg_features.shape}")  # 예상: (batch, 512, 7, 7)

        vgg_features = self.vgg_pool(vgg_features)
        if self.print_shapes:
            print(f"VGG16 pooled: {vgg_features.shape}")  # 예상: (batch, 512, 7, 7)

        vgg_features = self.vgg_conv1x1(vgg_features)
        if self.print_shapes:
            print(f"VGG16 1x1 Conv: {vgg_features.shape}")  # 예상: (batch, 1024, 7, 7)

        # ✅ Feature Concatenation
        fused_features = torch.cat([eff_features, vgg_features], dim=1)
        if self.print_shapes:
            print(f"Fused features: {fused_features.shape}")  # 예상: (batch, 2048, 7, 7)

        # ✅ ASPP 적용
        aspp_features = self.aspp(fused_features)
        if self.print_shapes:
            print(f"ASPP output: {aspp_features.shape}")  # 예상: (batch, 256, 7, 7)

        # ✅ Flatten 후 Classifier 적용
        aspp_features = aspp_features.view(aspp_features.size(0), -1)
        if self.print_shapes:
            print(f"Flattened: {aspp_features.shape}")  # 예상: (batch, 256*7*7)

        output = self.classifier(aspp_features)
        if self.print_shapes:
            print(f"[Output] {output.shape}")  # ✅ 최종 출력 크기 (batch, num_classes)

        return output


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 모델 생성 (print_shapes=True 설정)
model = EfficientNetVGGASPPHybrid(num_classes=6, print_shapes=True).to(device)

# ✅ 더미 입력 데이터 생성 (Batch=1, RGB=3, 224x224)
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# ✅ 모델 실행 (텐서 크기 출력)
output = model(dummy_input)



Loaded pretrained weights for efficientnet-b0

[Input] torch.Size([1, 3, 224, 224])
EfficientNet output: torch.Size([1, 1280, 7, 7])
EfficientNet pooled: torch.Size([1, 1280, 7, 7])
EfficientNet 1x1 Conv: torch.Size([1, 1024, 7, 7])
VGG16 output: torch.Size([1, 512, 7, 7])
VGG16 pooled: torch.Size([1, 512, 7, 7])
VGG16 1x1 Conv: torch.Size([1, 1024, 7, 7])
Fused features: torch.Size([1, 2048, 7, 7])
ASPP output: torch.Size([1, 256, 7, 7])
Flattened: torch.Size([1, 12544])
[Output] torch.Size([1, 6])


In [2]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# EfficientNet (pip install efficientnet-pytorch)
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm  # 학습 진행률 표시


# ==========================
# ✅ SEED 설정 (재현성 확보)
# ==========================
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==========================
# ✅ ASPP Block 정의 (Global Branch 포함)
# ==========================
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        # dilation = 1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        # dilation = 1
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        # dilation = 6
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        # dilation = 12
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        
        # Global Average Pool + 채널 변환
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        # ⭐ global_avg도 256채널로 맞춰주는 Conv
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)

        # 최종적으로 5가지(conv1, conv3_1, conv3_6, conv3_12, global_avg)를 concat → out_channels * 5
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        # (batch, in_channels, 7, 7)
        global_avg = self.global_avg_pool(x)
        # -> (batch, in_channels, 1, 1)
        global_avg = self.conv1_for_global(global_avg)
        # -> (batch, 256, 1, 1)
        global_avg = nn.functional.interpolate(
            global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True
        )
        # -> (batch, 256, 7, 7)

        # concat (256 each → 5 * 256 = 1280)
        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        out = self.conv1x1_out(out)  # (batch, 256, 7, 7)
        return out

# ==========================
# ✅ EfficientNet + VGG + ASPP 모델 정의
# ==========================
class EfficientNetVGGASPPHybrid(nn.Module):
    def __init__(self, num_classes=6):
        super(EfficientNetVGGASPPHybrid, self).__init__()

        # ✅ EfficientNet-b0
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()

        # ✅ VGG16
        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        # ✅ Feature Map 크기
        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  # VGG16 마지막 Conv Layer 출력

        # ✅ 7x7로 맞춤
        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        # ✅ 1x1 Conv로 채널 맞춤
        # 1280 → 1024
        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        # 512 → 1024
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        # ✅ 두 feature 합치면 2048 → ASPP
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        # ✅ Classifier
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # ✅ EfficientNet features
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)     # (batch, 1280, 7, 7)
        eff_features = self.eff_conv1x1(eff_features)           # (batch, 1024, 7, 7)

        # ✅ VGG16 features
        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)              # (batch, 512, 7, 7)
        vgg_features = self.vgg_conv1x1(vgg_features)           # (batch, 1024, 7, 7)

        # ✅ Feature Concatenation
        fused_features = torch.cat([eff_features, vgg_features], dim=1)  # (batch, 2048, 7, 7)

        # ✅ ASPP
        aspp_features = self.aspp(fused_features)               # (batch, 256, 7, 7)

        # ✅ Flatten & Classifier
        aspp_features = aspp_features.view(aspp_features.size(0), -1)  # (batch, 256*7*7)
        output = self.classifier(aspp_features)                       # (batch, num_classes)
        return output


# ==========================
# ✅ 데이터 로딩
# ==========================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# 사용자 환경에 맞춰 수정
dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# ==========================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetVGGASPPHybrid(num_classes=6).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ==========================
# ✅ 학습 함수
# ==========================
def train_one_epoch(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# ==========================
# ✅ 평가 함수
# ==========================
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# ==========================
# ✅ 학습 루프 (Best 모델 저장)
# ==========================
best_acc = 0.0
num_epochs = 50

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
          f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Best model 저장
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

print(f"\nBest Accuracy: {best_acc:.4f}")


Loaded pretrained weights for efficientnet-b0

[Epoch 1/50]


Train Loss: 3.7695 | Train Acc: 0.4110 | Val Loss: 2.7277 | Val Acc: 0.4237
  [*] Best model saved.

[Epoch 2/50]


Train Loss: 1.2522 | Train Acc: 0.6610 | Val Loss: 1.3098 | Val Acc: 0.6667
  [*] Best model saved.

[Epoch 3/50]


Train Loss: 0.9038 | Train Acc: 0.7938 | Val Loss: 1.0393 | Val Acc: 0.7797
  [*] Best model saved.

[Epoch 4/50]


Train Loss: 0.7962 | Train Acc: 0.8814 | Val Loss: 1.0518 | Val Acc: 0.7571

[Epoch 5/50]


...

Train Loss: 0.5133 | Train Acc: 0.9901 | Val Loss: 0.8181 | Val Acc: 0.8701

[Epoch 31/50]


Train Loss: 0.5061 | Train Acc: 0.9929 | Val Loss: 0.7390 | Val Acc: 0.8927

[Epoch 32/50]


Train Loss: 0.5070 | Train Acc: 0.9915 | Val Loss: 0.8403 | Val Acc: 0.8475

[Epoch 33/50]


Train Loss: 0.5171 | Train Acc: 0.9915 | Val Loss: 0.7801 | Val Acc: 0.8814

[Epoch 34/50]


KeyboardInterrupt: 

In [15]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm  # 학습 진행률 표시

# ==========================
# ✅ SEED 설정 (재현성 확보)
# ==========================
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        # dilation = 1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        # dilation = 1
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        # dilation = 6
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        # dilation = 12
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        # Global Average Pool
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # 최종적으로 5가지(conv1, conv3_1, conv3_6, conv3_12, global_avg)를 concat → out_channels * 5
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv3_1 = self.conv3_1(x)
        conv3_6 = self.conv3_6(x)
        conv3_12 = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        # 원래 feature 크기에 맞춰 upsample
        global_avg = nn.functional.interpolate(global_avg, size=conv3_1.shape[2:], 
                                               mode='bilinear', align_corners=True)

        out = torch.cat([conv1, conv3_1, conv3_6, conv3_12, global_avg], dim=1)
        out = self.conv1x1_out(out)
        return out


class EfficientNetVGGASPPHybrid(nn.Module):
    def __init__(self, num_classes=6):
        super(EfficientNetVGGASPPHybrid, self).__init__()

        # EfficientNet-b0
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        # 기존 FC 제거
        self.efficientnet._fc = nn.Identity()

        # VGG16 (PyTorch 1.12+)
        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        # 기존 FC 제거
        self.vgg.classifier = nn.Identity()

        # Feature map 채널 수
        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 보통 1280
        vgg_out_features = 512  # VGG16 마지막 Conv 블록 출력 채널

        # (h, w)를 7x7로 맞춤
        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        # 각각 1x1 Conv로 변환
        # 1280 → 1024
        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        # 512 → 1024
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        # ASPP: in_channels=2048 (1024 + 1024)
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # EfficientNet features
        eff_features = self.efficientnet.extract_features(x)   
        # -> 보통 (batch, 1280, h, w)
        eff_features = self.efficientnet_pool(eff_features)     
        # -> (batch, 1280, 7, 7)
        eff_features = self.eff_conv1x1(eff_features)          
        # -> (batch, 1024, 7, 7)

        # VGG16 features
        vgg_features = self.vgg.features(x) 
        # -> (batch, 512, h', w')
        vgg_features = self.vgg_pool(vgg_features) 
        # -> (batch, 512, 7, 7)
        vgg_features = self.vgg_conv1x1(vgg_features)
        # -> (batch, 1024, 7, 7)

        # Concat
        fused_features = torch.cat([eff_features, vgg_features], dim=1) 
        # -> (batch, 2048, 7, 7)

        # ASPP
        aspp_features = self.aspp(fused_features)  
        # -> (batch, 256, 7, 7)

        # Flatten
        aspp_features = aspp_features.view(aspp_features.size(0), -1)
        # -> (batch, 256*7*7)

        # Classifier
        output = self.classifier(aspp_features)
        # -> (batch, num_classes)

        return output


# ==========================
# ✅ 데이터 로딩
# ==========================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# ==========================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetVGGASPPHybrid(num_classes=6).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ==========================
# ✅ 학습 함수
# ==========================
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ==========================
# ✅ 평가 함수
# ==========================
def evaluate(model, test_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ==========================
# ✅ 학습 루프 (Best 모델 저장)
# ==========================
best_acc = 0.0
for epoch in range(50):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "best_model.pth")  # 모델 저장

    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")

print(f"Best Accuracy: {best_acc:.4f}")


Loaded pretrained weights for efficientnet-b0


Training:   0%|          | 0/45 [00:01<?, ?it/s]


RuntimeError: Given groups=1, weight of size [256, 1280, 1, 1], expected input[16, 3072, 7, 7] to have 1280 channels, but got 3072 channels instead